# * VINSIGHT : Gross Adds

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Import Transaction

In [3]:
''' Execute transaction '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250101
print(f'\nParameter input...\n')
print(f'   -> Data as of {curr_datetime}')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD, AREA_NAME
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL_SNAP END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
        , MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B1S000101CS' --Prepaid Gross Adds
        , 'TB1S000101CS' --Prepaid Gross Adds : TMH
        , 'DB1S000101CS' --Prepaid Gross Adds : DTAC
        , 'B1S000101CG' --Prepaid Gross Adds - GEO Channel
        , 'TB1S000101CG' --Prepaid Gross Adds : TMH - GEO Channel
        , 'DB1S000101CG' --Prepaid Gross Adds : DTAC - GEO Channel
        
        , 'B2S010100CS' --Postpaid Gross Adds B2C
        , 'TB2S010100CS' --Postpaid Gross Adds B2C : TMH
        , 'DB2S010100CS' --Postpaid Gross Adds B2C : DTAC
        , 'B2S020100CS' --Postpaid Gross Adds B2B
        , 'TB2S020100CS' --Postpaid Gross Adds B2B : TMH
        , 'DB2S020100CS' --Postpaid Gross Adds B2B : DTAC
        , 'B2S010100CG' --Postpaid Gross Adds B2C - GEO Channel
        , 'TB2S010100CG' --Postpaid Gross Adds B2C : TMH - GEO Channel
        , 'DB2S010100CG' --Postpaid Gross Adds B2C : DTAC - GEO Channel
        , 'B2S020100CG' --Postpaid Gross Adds B2B - GEO Channel
        , 'TB2S020100CG' --Postpaid Gross Adds B2B : TMH - GEO Channel
        , 'DB2S020100CG' --Postpaid Gross Adds B2B : DTAC - GEO Channel
        
        , 'TB3S000102CS' --TOL Gross Adds Connected : Consumer
        , 'TB3S000102CG' --TOL Gross Adds Connected : Consumer - GEO Channel
        , 'TB3S000102D1CS' --TOL Gross Adds Connected : Consumer (Install Location)
        , 'TB3S000102D1CG' --TOL Gross Adds Connected : Consumer - GEO Channel (Install Location)
        , 'TB3S000103CS' --TOL Gross Adds Connected : DataService
        , 'TB3S000103CG' --TOL Gross Adds Connected : DataService - GEO Channel
        , 'TB3S000103D1CS' --TOL Gross Adds Connected : DataService (Install Location)
        , 'TB3S000103D1CG' --TOL Gross Adds Connected : DataService - GEO Channel (Install Location)
        , 'TB3S000104CS' --TOL Gross Adds Registration : Consumer
        , 'TB3S000104CG' --TOL Gross Adds Registration : Consumer - GEO Channel
        , 'TB3S000105CS' --TOL Gross Adds Registration : DataService
        , 'TB3S000105CG' --TOL Gross Adds Registration : DataService - GEO Channel
        
        , 'TB4S001004CS' --TVS CMDU Gross Adds
        , 'TB4S001004CG' --TVS CMDU Gross Adds - GEO Channel
        , 'TB4S001400CS' --TVS Now Gross Adds
        , 'TB4S001400CG' --TVS Now Gross Adds - GEO Channel
        )
    AND AREA_TYPE IN ('C','P','G','H','HH')
    --AND TM_KEY_DAY IN (20240131, 20240229, 20240331, 20240430, 20240531, 20240630, 20240731, 20240831, 20240930, 20241031, 20241130, 20241231, 20250131, 20250228, 20250331) -->> Year 2024
    --AND TM_KEY_DAY IN (20250131, 20250228, 20250331, 20250430, 20250531) -->> Year 2025
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
    --ORDER BY TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD
""")


try:
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...

   -> Data as of 2025-06-03, 16:06:20
   -> v_start_date: 20250101


In [4]:
''' Automate Currently Period '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = str(chk_src_df['TM_KEY_MTH'].drop_duplicates().astype(int).shift().max())

## Daily (All & Geo)

In [14]:
''' Gross Adds(All Channel) Daily '''

v_metric_list = [
	'B1S000101CS' #Prepaid Gross Adds
	, 'TB1S000101CS' #Prepaid Gross Adds : TMH
	, 'DB1S000101CS' #Prepaid Gross Adds : DTAC
	
	, 'B2S010100CS' #Postpaid Gross Adds B2C
	, 'TB2S010100CS' #Postpaid Gross Adds B2C : TMH
	, 'DB2S010100CS' #Postpaid Gross Adds B2C : DTAC
	# , 'B2S020100CS' #Postpaid Gross Adds B2B
	# , 'TB2S020100CS' #Postpaid Gross Adds B2B : TMH
	# , 'DB2S020100CS' #Postpaid Gross Adds B2B : DTAC
	
	, 'TB3S000104CS' #TOL Gross Adds Registration : Consumer	
	, 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
	# , 'TB3S000102D1CS' #TOL Gross Adds Connected : Consumer (Install Location)
    # , 'TB3S000105CS' #TOL Gross Adds Registration : DataService
	# , 'TB3S000103CS' #TOL Gross Adds Connected : DataService
	# , 'TB3S000103D1CS' #TOL Gross Adds Connected : DataService (Install Location)
	
    , 'TB4S001400CS' #TVS Now Gross Adds
	# , 'TB4S001004CS' #TVS CMDU Gross Adds
    ]

# v_tm_key_mth = curr_mth
v_tm_key_mth = 202505

ga_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
ga_daily_df = ga_daily_df.loc[ga_daily_df['METRIC_CD'].isin(v_metric_list)]
ga_daily_df = ga_daily_df.loc[ga_daily_df['TM_KEY_MTH']>=v_tm_key_mth]

ga_daily_df['PRE'] = np.where(ga_daily_df['METRIC_CD']=='B1S000101CS', ga_daily_df['P'], 0)
ga_daily_df['PRE_T'] = np.where(ga_daily_df['METRIC_CD']=='TB1S000101CS', ga_daily_df['P'], 0)
ga_daily_df['PRE_D'] = np.where(ga_daily_df['METRIC_CD']=='DB1S000101CS', ga_daily_df['P'], 0)
ga_daily_df['POST_B2C'] = np.where(ga_daily_df['METRIC_CD']=='B2S010100CS', ga_daily_df['P'], 0)
ga_daily_df['POST_B2C_T'] = np.where(ga_daily_df['METRIC_CD']=='TB2S010100CS', ga_daily_df['P'], 0)
ga_daily_df['POST_B2C_D'] = np.where(ga_daily_df['METRIC_CD']=='DB2S010100CS', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B'] = np.where(ga_daily_df['METRIC_CD']=='B2S020100CS', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_T'] = np.where(ga_daily_df['METRIC_CD']=='TB2S020100CS', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_D'] = np.where(ga_daily_df['METRIC_CD']=='DB2S020100CS', ga_daily_df['P'], 0)
ga_daily_df['TOL_REG_CON'] = np.where(ga_daily_df['METRIC_CD']=='TB3S000104CS', ga_daily_df['P'], 0)
ga_daily_df['TOL_CONN'] = np.where(ga_daily_df['METRIC_CD']=='TB3S000102CS', ga_daily_df['P'], 0)
# ga_daily_df['TOL_CONN(Install)'] = np.where(ga_daily_df['METRIC_CD']=='TB3S000102D1CS', ga_daily_df['P'], 0)
# ga_daily_df['TOL_REG_DSV'] = np.where(ga_daily_df['METRIC_CD']=='TB3S000105CS', ga_daily_df['P'], 0)
# ga_daily_df['TOL_DSV'] = np.where(ga_daily_df['METRIC_CD']=='TB3S000103CS', ga_daily_df['P'], 0)
# ga_daily_df['TOL_DSV(Install)'] = np.where(ga_daily_df['METRIC_CD']=='TB3S000103D1CS', ga_daily_df['P'], 0)
ga_daily_df['TVS_NOW'] = np.where(ga_daily_df['METRIC_CD']=='TB4S001400CS', ga_daily_df['P'], 0)
# ga_daily_df['TVS_CMDU'] = np.where(ga_daily_df['METRIC_CD']=='TB4S001004CS', ga_daily_df['P'], 0)

# ga_daily_df = ga_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_REG_DSV':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# ga_daily_df = ga_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY']).reset_index()
# ga_daily_df = ga_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_REG_CON', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_REG_DSV', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

ga_daily_df = ga_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
ga_daily_df = ga_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY']).reset_index()
ga_daily_df = ga_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_REG_CON', 'TOL_CONN', 'TVS_NOW']]

mod_col_list = ga_daily_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_daily_df[col] = ga_daily_df[col].apply(lambda x: format(x, ',.0f'))
ga_daily_df

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_REG_CON,TOL_CONN,TVS_NOW
0,202505,20250501,2025-06-03 03:20:35,"86,416","38,901","47,515","6,692","4,418","2,274","1,945",72,136
1,202505,20250502,2025-06-03 03:20:35,"71,701","34,737","36,964","6,422","4,936","1,486","2,528","1,860",105
2,202505,20250503,2025-06-03 03:20:35,"60,870","30,845","30,025","6,618","5,251","1,367","2,488","1,982",92
3,202505,20250504,2025-06-03 03:20:35,"56,850","27,906","28,944","6,592","5,600",992,"2,118","1,803",137
4,202505,20250505,2025-06-03 03:20:35,"58,767","29,305","29,462","6,435","4,829","1,606","2,534","1,908",162
5,202505,20250506,2025-06-03 03:20:35,"62,884","30,006","32,878","7,217","5,306","1,911","2,753","1,951",66
6,202505,20250507,2025-06-03 03:20:35,"64,183","32,026","32,157","7,743","5,919","1,824","2,618","1,882",66
7,202505,20250508,2025-06-03 03:20:35,"61,544","31,666","29,878","6,296","4,635","1,661","2,827","1,857",74
8,202505,20250509,2025-06-03 03:20:35,"63,617","32,060","31,557","6,331","4,438","1,893","2,600","1,903",84
9,202505,20250510,2025-06-03 03:20:35,"62,911","30,098","32,813","5,935","4,232","1,703","2,341","1,676",103


In [15]:
''' Gross Adds(Geo Channel) Daily '''

v_metric_list = [
	'B1S000101CG' #Prepaid Gross Adds - GEO Channel
	, 'TB1S000101CG' #Prepaid Gross Adds : TMH - GEO Channel
	, 'DB1S000101CG' #Prepaid Gross Adds : DTAC - GEO Channel
	
	, 'B2S010100CG' #Postpaid Gross Adds B2C - GEO Channel
	, 'TB2S010100CG' #Postpaid Gross Adds B2C : TMH - GEO Channel
	, 'DB2S010100CG' #Postpaid Gross Adds B2C : DTAC - GEO Channel
	# , 'B2S020100CG' #Postpaid Gross Adds B2B - GEO Channel
	# , 'TB2S020100CG' #Postpaid Gross Adds B2B : TMH - GEO Channel
	# , 'DB2S020100CG' #Postpaid Gross Adds B2B : DTAC - GEO Channel

	, 'TB3S000104CG' #TOL Gross Adds Registration : Consumer - GEO Channel	
	, 'TB3S000102CG' #TOL Gross Adds Connected : Consumer - GEO Channel
	# , 'TB3S000102D1CG' #TOL Gross Adds Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3S000105CG' #TOL Gross Adds Registration : DataService - GEO Channel
	# , 'TB3S000103CG' #TOL Gross Adds Connected : DataService - GEO Channel
	# , 'TB3S000103D1CG' #TOL Gross Adds Connected : DataService - GEO Channel (Install Location)
	
	, 'TB4S001400CG' #TVS Now Gross Adds - GEO Channel
	# , 'TB4S001004CG' #TVS CMDU Gross Adds - GEO Channel
    ]

# v_tm_key_mth = curr_mth
v_tm_key_mth = 202505

ga_geo_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
ga_geo_daily_df = ga_geo_daily_df.loc[ga_geo_daily_df['METRIC_CD'].isin(v_metric_list)]
ga_geo_daily_df = ga_geo_daily_df.loc[ga_geo_daily_df['TM_KEY_MTH']>=v_tm_key_mth]

ga_geo_daily_df['PRE'] = np.where(ga_geo_daily_df['METRIC_CD']=='B1S000101CG', ga_geo_daily_df['P'], 0)
ga_geo_daily_df['PRE_T'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB1S000101CG', ga_geo_daily_df['P'], 0)
ga_geo_daily_df['PRE_D'] = np.where(ga_geo_daily_df['METRIC_CD']=='DB1S000101CG', ga_geo_daily_df['P'], 0)
ga_geo_daily_df['POST_B2C'] = np.where(ga_geo_daily_df['METRIC_CD']=='B2S010100CG', ga_geo_daily_df['P'], 0)
ga_geo_daily_df['POST_B2C_T'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB2S010100CG', ga_geo_daily_df['P'], 0)
ga_geo_daily_df['POST_B2C_D'] = np.where(ga_geo_daily_df['METRIC_CD']=='DB2S010100CG', ga_geo_daily_df['P'], 0)
# ga_daily_df['POST_B2B'] = np.where(ga_daily_df['METRIC_CD']=='B2S020100CG', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_T'] = np.where(ga_daily_df['METRIC_CD']=='TB2S020100CG', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_D'] = np.where(ga_daily_df['METRIC_CD']=='DB2S020100CG', ga_daily_df['P'], 0)
ga_geo_daily_df['TOL_REG_CON'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB3S000104CG', ga_geo_daily_df['P'], 0)
ga_geo_daily_df['TOL_CONN'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB3S000102CG', ga_geo_daily_df['P'], 0)
# ga_geo_daily_df['TOL_CONN(Install)'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB3S000102D1CG', ga_geo_daily_df['P'], 0)
# ga_geo_daily_df['TOL_REG_DSV'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB3S000105CG', ga_geo_daily_df['P'], 0)
# ga_geo_daily_df['TOL_DSV'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB3S000103CG', ga_geo_daily_df['P'], 0)
# ga_geo_daily_df['TOL_DSV(Install)'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB3S000103D1CG', ga_geo_daily_df['P'], 0)
ga_geo_daily_df['TVS_NOW'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB4S001400CG', ga_geo_daily_df['P'], 0)
# ga_geo_daily_df['TVS_CMDU'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB4S001004CG', ga_geo_daily_df['P'], 0)

# ga_geo_daily_df = ga_geo_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_REG_DSV':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# ga_geo_daily_df = ga_geo_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY']).reset_index()
# ga_geo_daily_df = ga_geo_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_REG_CON', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_REG_DSV', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

ga_geo_daily_df = ga_geo_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
ga_geo_daily_df = ga_geo_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY']).reset_index()
ga_geo_daily_df = ga_geo_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_REG_CON', 'TOL_CONN', 'TVS_NOW']]

mod_col_list = ga_geo_daily_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_geo_daily_df[col] = ga_geo_daily_df[col].apply(lambda x: format(x, ',.0f'))
ga_geo_daily_df

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_REG_CON,TOL_CONN,TVS_NOW
0,202505,20250501,2025-06-03 03:20:35,"85,835","38,782","47,053","2,998","2,289",709,"1,554",62,136
1,202505,20250502,2025-06-03 03:20:35,"71,205","34,593","36,612","3,075","2,554",521,"2,047","1,520",105
2,202505,20250503,2025-06-03 03:20:35,"60,408","30,714","29,694","3,033","2,596",437,"1,987","1,562",92
3,202505,20250504,2025-06-03 03:20:35,"56,458","27,819","28,639","3,255","2,672",583,"1,673","1,398",137
4,202505,20250505,2025-06-03 03:20:35,"58,414","29,230","29,184","2,820","2,347",473,"1,906","1,536",160
5,202505,20250506,2025-06-03 03:20:35,"62,474","29,882","32,592","3,184","2,693",491,"2,128","1,621",66
6,202505,20250507,2025-06-03 03:20:35,"63,710","31,847","31,863","3,456","2,916",540,"1,985","1,540",66
7,202505,20250508,2025-06-03 03:20:35,"61,033","31,461","29,572","2,665","2,235",430,"2,173","1,498",74
8,202505,20250509,2025-06-03 03:20:35,"63,201","31,890","31,311","2,525","2,095",430,"2,081","1,545",84
9,202505,20250510,2025-06-03 03:20:35,"62,512","29,961","32,551","2,443","2,033",410,"1,818","1,280",103


In [ ]:
	# 'B1S000101CS' #Prepaid Gross Adds
	# , 'TB1S000101CS' #Prepaid Gross Adds : TMH
	# , 'DB1S000101CS' #Prepaid Gross Adds : DTAC
	# , 'B1S000101CG' #Prepaid Gross Adds - GEO Channel
	# , 'TB1S000101CG' #Prepaid Gross Adds : TMH - GEO Channel
	# , 'DB1S000101CG' #Prepaid Gross Adds : DTAC - GEO Channel
	
	# , 'B2S010100CS' #Postpaid Gross Adds B2C
	# , 'TB2S010100CS' #Postpaid Gross Adds B2C : TMH
	# , 'DB2S010100CS' #Postpaid Gross Adds B2C : DTAC
	# , 'B2S020100CS' #Postpaid Gross Adds B2B
	# , 'TB2S020100CS' #Postpaid Gross Adds B2B : TMH
	# , 'DB2S020100CS' #Postpaid Gross Adds B2B : DTAC
	# , 'B2S010100CG' #Postpaid Gross Adds B2C - GEO Channel
	# , 'TB2S010100CG' #Postpaid Gross Adds B2C : TMH - GEO Channel
	# , 'DB2S010100CG' #Postpaid Gross Adds B2C : DTAC - GEO Channel
	# , 'B2S020100CG' #Postpaid Gross Adds B2B - GEO Channel
	# , 'TB2S020100CG' #Postpaid Gross Adds B2B : TMH - GEO Channel
	# , 'DB2S020100CG' #Postpaid Gross Adds B2B : DTAC - GEO Channel
	
	# , 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
	# , 'TB3S000102CG' #TOL Gross Adds Connected : Consumer - GEO Channel
	# , 'TB3S000102D1CS' #TOL Gross Adds Connected : Consumer (Install Location)
	# , 'TB3S000102D1CG' #TOL Gross Adds Connected : Consumer - GEO Channel (Install Location)
	# , 'TB3S000103CS' #TOL Gross Adds Connected : DataService
	# , 'TB3S000103CG' #TOL Gross Adds Connected : DataService - GEO Channel
	# , 'TB3S000103D1CS' #TOL Gross Adds Connected : DataService (Install Location)
	# , 'TB3S000103D1CG' #TOL Gross Adds Connected : DataService - GEO Channel (Install Location)
	# , 'TB3S000104CS' #TOL Gross Adds Registration : Consumer
	# , 'TB3S000104CG' #TOL Gross Adds Registration : Consumer - GEO Channel
	# , 'TB3S000105CS' #TOL Gross Adds Registration : DataService
	# , 'TB3S000105CG' #TOL Gross Adds Registration : DataService - GEO Channel
	
	# , 'TB4S001004CS' #TVS CMDU Gross Adds
	# , 'TB4S001004CG' #TVS CMDU Gross Adds - GEO Channel
	# , 'TB4S001400CS' #TVS Now Gross Adds
	# , 'TB4S001400CG' #TVS Now Gross Adds - GEO Channel